In [2]:
import pandas as pd
import geopandas as gpd
from shapely import Point
from math import radians, cos, sin, asin, sqrt

In [3]:
RADIUS_KM = 1.7

In [4]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    # print("Distance:", c * r)
    return c * r

In [5]:
"""
funtion will take 4 inputs. infogroup_df & counterglow_df2 are the dataframes being analyzed.
the third input is the specific state that we are choosing to analyze
the fourth input is the threshold distance. this distance is the max allowable distance between 2
datapoints. the output is a dataframe that in each row, we have a datapoint that represents a
datapoint from infogroup and (x distance away) a farm spotted in the counterglow data. we are
speculating that it is possible for a spotted farm in counterglow to not have the most precise address
and actually exist in the Infogroup data, but with slightly different longitude and latitude
"""

def potential_farms(infogroup_df, counterglow_df, two_letter_state, max_dist_km ):
    
    # ruduce dataframe to only consist rows with specified two_letter_state
    infogroup_state_df      = infogroup_df[infogroup_df["STATE"] == two_letter_state]
    counterglow_state_df    = counterglow_df[counterglow_df["State"] == two_letter_state]
    
    # create two new empty dataframe that we will be adding to when match is found
    infogroup_trim     = pd.DataFrame(columns=infogroup_state_df.columns)
    counterglow_trim   = pd.DataFrame(columns=counterglow_state_df.columns)
    
    counter = 0 # to counter matches, can print to terminal if desired
    for i in range(len(infogroup_state_df)):
        
        infogroup_longitude  = infogroup_state_df.iloc[i]["LONGITUDE"]  # get longitude
        infogroup_latitude   = infogroup_state_df.iloc[i]["LATITUDE"]   # get latitude
        
        for j in range(len(counterglow_state_df)):
            
            counterglow_latitude    = counterglow_state_df.iloc[j]["Lat"]    # get latitude
            counterglow_longitude   = counterglow_state_df.iloc[j]["Lat.1"]  # get longitude
            
            dist_km                 = haversine(infogroup_longitude, infogroup_latitude, 
                                                counterglow_longitude, counterglow_latitude)
            
            if(dist_km <= max_dist_km): # if the distance is <= the max distance set in function
                counter += 1
            
                infogroup_trim.loc[len(infogroup_trim.index)]      = infogroup_state_df.iloc[i]
                counterglow_trim.loc[len(counterglow_trim.index)]  = counterglow_state_df.iloc[j]
        
    # change the name of the columns for Latitude & Longitude in each dataframe
    # for Infogroup Dataframe
    infogroup_trim      = infogroup_trim.rename(columns={'LATITUDE': 'LATITUDE 1', 
                                                        'LONGITUDE': 'LONGITUDE 1'})
    counterglow_trim    = counterglow_trim.rename(columns={'Lat' : 'LATITUDE 2',
                                                           'Lat.1': 'LONGITUDE 2'})
    
    if(counter !=0):
        print(two_letter_state + ":", "Total number of matches:", counter)
                
    return infogroup_trim, counterglow_trim   
    

In [6]:
"""
function take a dataframe and a list of states. This function calls the function 'potential_farms'
if the two dataframes returned from 'potential_farms' are NOT empty, they are added to the dictionaries
the keys for the dictionaries are the STATE in the current iteration (in the for loop) and the values
are the dataframes associated with the state. This function will return two dictionaries
"""

def infogroup_counterglow_dict(sic_df, counterglow_df, list_of_states, radium_km):
    # empty dictionaries
    igroup_sic_dict = {}
    ctrglow_dict = {}

    # loop through states list and use those as keys to match with the dataframe that was output 
    for state in range(len(list_of_states)):
        infogroup_matches, counterglow_matches = potential_farms(sic_df, counterglow_df, list_of_states[state], radium_km)
        
        # set up key value pairing for both dictionaries
        # only adding to dictionaries if there are matches
        if( (len(infogroup_matches)!=0) & (len(counterglow_matches)!=0) ):
            igroup_sic_dict[list_of_states[state]]    = infogroup_matches
            ctrglow_dict[list_of_states[state]]       = counterglow_matches
            
    return igroup_sic_dict, ctrglow_dict


In [7]:
"""
fucntion will take a dataframe and trim off all columns, only leaving:
Company, address line 1, city, state, zipcode, primary sic code, sic code 1, sic code 2, sic code 3, sic code 4,
"""

def keep_certain_columns(dict):
    for key in dict:
        dict[key] = dict[key][['COMPANY', 'ADDRESS LINE 1', 'CITY', 'STATE', 'ZIPCODE', 'PRIMARY SIC CODE', 'SIC6_DESCRIPTIONS',
                'PRIMARY NAICS CODE', 'NAICS8 DESCRIPTIONS', 'SIC CODE', 'SIC6_DESCRIPTIONS (SIC)',
                'SIC CODE 1', 'SIC6_DESCRIPTIONS (SIC1)', 'SIC CODE 2', 'SIC6_DESCRIPTIONS(SIC2)',
                'SIC CODE 3', 'SIC6_DESCRIPTIONS(SIC3)', 'SIC CODE 4', 'SIC6_DESCRIPTIONS(SIC4)', 'PARENT NUMBER']]
    
    return dict


In [8]:
# load counterglow dataset
df_counterglow = pd.read_csv("../data/Counterglow+Facility+List+Complete.csv")

In [9]:
# filter counterglow dataset to have poultry/chicken only

# Chickens & Other Birds (Meat)
# Chickens (Meat)

df_counterglow_poultry = df_counterglow[(df_counterglow['Farm Type'] == 'Chickens & Other Birds (Meat)') |
                                        (df_counterglow['Farm Type'] == 'Chickens (Meat)')]


In [10]:
states = ['IA', 'OK', 'MO', 'OK', 'AL', 'LA', 'MS', 'IL', 'IN', 'OH', 'KY', 'TN', 'AR', 'NC', 'SC', 'GA']

## SIC Code 0291 Exploration: General Farms, Primarily Livestock and Animal Specialties.

In [60]:
# To see how many matches exist within the dataframe consisting of SIC Codes containing 0291
df_sic_0291 = pd.read_csv("../data/code_0291.csv")
  
igroup_sic_0291_dict, ctrglow_0291_dict = infogroup_counterglow_dict(df_sic_0291, df_counterglow_poultry, states, RADIUS_KM)


MO: Total number of matches: 1
AL: Total number of matches: 2
NC: Total number of matches: 1


In [63]:
igroup_sic_0291_dict = keep_certain_columns(igroup_sic_0291_dict)

#### Explore

In [64]:
igroup_sic_0291_dict['MO']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,K-MAC RANCH,15223 STATE HIGHWAY 37,CASSVILLE,MO,65625,29101,RANCHES,11299013.0,ALL OTHER ANIMAL PRODUCTION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
igroup_sic_0291_dict['AL']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,RAEANN RANCH LLC,2665 KELLY CREEK RD,ODENVILLE,AL,35120,29101,RANCHES,11299013.0,ALL OTHER ANIMAL PRODUCTION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NARROW GATE RANCH,5652 BIRD FARM RD,JASPER,AL,35503,29101,RANCHES,11299013.0,ALL OTHER ANIMAL PRODUCTION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
igroup_sic_0291_dict['NC']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,LATTAS EGG RANCH INC,1016 GOVERNOR BURKE RD,HILLSBOROUGH,NC,27278,29101,RANCHES,11299013.0,ALL OTHER ANIMAL PRODUCTION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## SIC Code 0241 Exploration: Dairy Farms

In [67]:
# To see how many matches exist within the dataframe consisting of SIC Codes containing 0241
df_sic_0241 = pd.read_csv("../data/code_0241.csv")
  
igroup_sic_0241_dict, ctrglow_0241_dict = infogroup_counterglow_dict(df_sic_0241, df_counterglow_poultry, states, RADIUS_KM)


IA: Total number of matches: 1
MS: Total number of matches: 1
IN: Total number of matches: 3
OH: Total number of matches: 2
NC: Total number of matches: 3


In [69]:
igroup_sic_0241_dict = keep_certain_columns(igroup_sic_0241_dict)

#### Explore

In [71]:
igroup_sic_0241_dict['IA']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,WIELENGA DAIRY INC,3387 JAY AVE,BOYDEN,IA,51234,24101,DAIRIES (MILK),11212001.0,DAIRY CATTLE & MILK PRODUCTION,545101.0,DAIRY PRODUCTS-RETAIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
igroup_sic_0241_dict['MS']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,BENNETT DAIRY INC,1173 KOKOMO RD,KOKOMO,MS,39643,24101,DAIRIES (MILK),11212001.0,DAIRY CATTLE & MILK PRODUCTION,999966.0,FEDERAL GOVERNMENT CONTRACTORS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
igroup_sic_0241_dict['IN']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,LA DON FARMS,13764 COUNTY ROAD 28,MIDDLEBURY,IN,46540,24103,DAIRY FARMS,11212002.0,DAIRY CATTLE & MILK PRODUCTION,19101.0,FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LA DON FARMS,13764 COUNTY ROAD 28,MIDDLEBURY,IN,46540,24103,DAIRY FARMS,11212002.0,DAIRY CATTLE & MILK PRODUCTION,19101.0,FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,THOMAS BONTRAGER FARM,63666 COUNTY ROAD 37,GOSHEN,IN,46528,19198,GENERAL FARMS-PRIMARILY CROP,11199807.0,ALL OTHER MISCELLANEOUS CROP FARMING,24103.0,DAIRY FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
igroup_sic_0241_dict['OH']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,R F & SONS,5629 STATE ROUTE 49,FORT RECOVERY,OH,45846,24103,DAIRY FARMS,11212002.0,DAIRY CATTLE & MILK PRODUCTION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MERCER COUNTY DAIRY ASSN INC,7274 INDIAN TRAIL RD,MARIA STEIN,OH,45860,24101,DAIRIES (MILK),11212001.0,DAIRY CATTLE & MILK PRODUCTION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
igroup_sic_0241_dict['NC']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,S & I RIVERSIDE DAIRY LLC,8317 SHULL DAIRY RD,VALE,NC,28168,24103,DAIRY FARMS,11212002.0,DAIRY CATTLE & MILK PRODUCTION,24101.0,DAIRIES (MILK),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,FOGLEMAN DAIRY,9144 SYLVAN RD,LIBERTY,NC,27298,24103,DAIRY FARMS,11212002.0,DAIRY CATTLE & MILK PRODUCTION,24101.0,DAIRIES (MILK),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,VIRGIL SHULL DAIRY FARM,5837 OLD SHELBY RD # 1,VALE,NC,28168,24101,DAIRIES (MILK),11212001.0,DAIRY CATTLE & MILK PRODUCTION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## SIC Code 0761 Exploration: Farm Labor

In [76]:
# To see how many matches exist within the dataframe consisting of SIC Codes containing 0761
df_sic_0761 = pd.read_csv("../data/code_0761.csv")
  
igroup_sic_0761_dict, ctrglow_0761_dict = infogroup_counterglow_dict(df_sic_0761, df_counterglow_poultry, states, RADIUS_KM)


MS: Total number of matches: 1


In [77]:
igroup_sic_0761_dict = keep_certain_columns(igroup_sic_0761_dict)

#### Explore

In [78]:
igroup_sic_0761_dict['MS']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,CHICKEN FARM,104 MISSISSIPPI 550,UNION CHURCH,MS,39668,76198,FARM LABOR,11511503.0,FARM LABOR CONTRACTORS & CREW LEADERS,76204.0,FARM MANAGEMENT SERVICE,581208.0,RESTAURANTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## SIC Code 0762 Exploration: Farm Mgmt Services

In [79]:
# To see how many matches exist within the dataframe consisting of SIC Codes containing 0762
df_sic_0762 = pd.read_csv("../data/code_0762.csv")
  
igroup_sic_0762_dict, ctrglow_0762_dict = infogroup_counterglow_dict(df_sic_0762, df_counterglow_poultry, states, RADIUS_KM)


IA: Total number of matches: 2
AL: Total number of matches: 2
MS: Total number of matches: 2
OH: Total number of matches: 1


In [80]:
igroup_sic_0762_dict = keep_certain_columns(igroup_sic_0762_dict)

#### Explore

In [81]:
igroup_sic_0762_dict['IA']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,FJELLAND REAL EST & FARM MGMT,203 ISABELLA ST,RADCLIFFE,IA,50230,653118,REAL ESTATE,53121003.0,OFFICES OF REAL ESTATE AGENTS & BROKERS,76204.0,FARM MANAGEMENT SERVICE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ASH SHADE AG SVC LLC,3970 300TH ST,BOYDEN,IA,51234,76203,FARMING SERVICE,11511606.0,FARM MANAGEMENT SERVICES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
igroup_sic_0762_dict['AL']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,ELLIS AG LLC,7265 AL HIGHWAY 9 S,CENTRE,AL,35960,76203,FARMING SERVICE,11511606.0,FARM MANAGEMENT SERVICES,422101.0,GRAIN ELEVATORS,526139.0,FERTILIZERS-RETAIL,519102.0,FARM SUPPLIES (WHLS),NaN,NaN,NaN,NaN,NaN
1,AMERICAN PROTEIN FARM MRTLTY,819 COUNTY ROAD 3328,BRUNDIDGE,AL,36010,76203,FARMING SERVICE,11511606.0,FARM MANAGEMENT SERVICES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
igroup_sic_0762_dict['MS']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,CHICKEN FARM,104 MISSISSIPPI 550,UNION CHURCH,MS,39668,76198,FARM LABOR,11511503.0,FARM LABOR CONTRACTORS & CREW LEADERS,76204.0,FARM MANAGEMENT SERVICE,581208.0,RESTAURANTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DAVID SHARP FARM,357 ROBERT BUTLER RD,FOREST,MS,39074,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,76203.0,FARMING SERVICE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
igroup_sic_0762_dict['OH']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,SUNRISE CO-OP,5458 STATE ROUTE 49,FORT RECOVERY,OH,45846,519111,FEED-WHOLESALE,42491013.0,FARM SUPPLIES MERCHANT WHOLESALERS,76203.0,FARMING SERVICE,519112.0,FEED-DEALERS (WHLS),NaN,NaN,NaN,NaN,NaN,NaN,5503818.0


## SIC Code 8748 Exploration: Farm Mgmt Systems

In [85]:
# To see how many matches exist within the dataframe consisting of SIC Codes containing 0762
df_sic_8748 = pd.read_csv("../data/code_8748.csv")
  
igroup_sic_8748_dict, ctrglow_8748_dict = infogroup_counterglow_dict(df_sic_8748, df_counterglow_poultry, states, RADIUS_KM)


IA: Total number of matches: 9
MO: Total number of matches: 2
AL: Total number of matches: 12
MS: Total number of matches: 4
IL: Total number of matches: 2
IN: Total number of matches: 3
OH: Total number of matches: 1
TN: Total number of matches: 1
AR: Total number of matches: 3
NC: Total number of matches: 14


In [86]:
igroup_sic_8748_dict = keep_certain_columns(igroup_sic_8748_dict)

#### Explore

In [87]:
igroup_sic_8748_dict['IA']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,FOWLER CONSULTING LLC,2425 240TH ST,DALLAS CENTER,IA,50063.0,871301,SURVEYORS-LAND,54137002.0,SURVEYING & MAPPING (EXCEPT GEOPHYSICAL) SERVICES,874899.0,CONSULTANTS-BUSINESS NEC,871301.0,SURVEYORS-LAND,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,RADCLIFFE TELEPHONE CO,202 ISABELLA ST,RADCLIFFE,IA,50230.0,481304,TELEPHONE COMPANIES,51711006.0,NaN,874823.0,COMMUNICATIONS CONSULTANTS,731103.0,ADVERTISING-DIRECTORY & GUIDE,481302.0,TELECOMMUNICATIONS SERVICES,NaN,NaN,NaN,NaN,NaN
2,HORIZON BEEF,216 MAIN ST,SANBORN,IA,51248.0,874820,AGRICULTURAL CONSULTANTS,54169010.0,OTHER SCIENTIFIC & TECHNICAL CONSULTING SERVICES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HARBOR GROUP INC,444 12TH ST NE,SIOUX CENTER,IA,51250.0,874899,CONSULTANTS-BUSINESS NEC,54161830.0,OTHER MANAGEMENT CONSULTING SERVICES,874201.0,BUSINESS MANAGEMENT CONSULTANTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NRCS,904 E TAYLOR ST,CRESTON,IA,50801.0,912101,GOVERNMENT OFFICES-FEDERAL,92112009.0,LEGISLATIVE BODIES,874820.0,AGRICULTURAL CONSULTANTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,ECHO GROUP INC,950 N MAIN AVE,SIOUX CENTER,IA,51250.0,874801,LIGHTING CONSULTANTS,54169031.0,OTHER SCIENTIFIC & TECHNICAL CONSULTING SERVICES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,556633493.0
6,JACKSON CATTLE INC,3568 JACKSON AVE,BOYDEN,IA,51234.0,874899,CONSULTANTS-BUSINESS NEC,54161830.0,OTHER MANAGEMENT CONSULTING SERVICES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,R & D DAIRY LLC,1366 210TH AVE,OSSIAN,IA,52161.0,874849,DAIRY CONSULTANTS,54169044.0,OTHER SCIENTIFIC & TECHNICAL CONSULTING SERVICES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NEW COOPERATIVE INC,12 HUMBOLDT AVE,BODE,IA,50519.0,422101,GRAIN ELEVATORS,49313003.0,FARM PRODUCT WAREHOUSING & STORAGE,519114.0,FERTILIZERS (WHLS),874820.0,AGRICULTURAL CONSULTANTS,519102.0,FARM SUPPLIES (WHLS),NaN,NaN,NaN,NaN,3512506.0


In [88]:
igroup_sic_8748_dict['MO']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,SOUTHWEST AVIATION & MFG LLC,164 ROUTE J,SOUTH WEST CITY,MO,64863.0,559905,AIRCRAFT-DEALERS,44122815.0,"MOTORCYCLE, ATV & ALL OTHER MOTOR VEHICLE DEALERS",874822.0,AVIATION CONSULTANTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SOUTHWEST AVIATION & MFG LLC,164 ROUTE J,SOUTH WEST CITY,MO,64863.0,559905,AIRCRAFT-DEALERS,44122815.0,"MOTORCYCLE, ATV & ALL OTHER MOTOR VEHICLE DEALERS",874822.0,AVIATION CONSULTANTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [89]:
igroup_sic_8748_dict['AL']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,WALMART TECH SVC,1450 N BRINDLEE MOUNTAIN PKWY,ARAB,AL,35016.0,874813,SECURITY SYSTEMS CONSULTANTS,54161813.0,OTHER MANAGEMENT CONSULTING SERVICES,599902.0,CELLULAR TELEPHONES-EQUIPMENT & SUPLS,737904.0,COMPUTERS-NETWORKING,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,RANDY DAVIS FARM,122 COUNTY ROAD 856,CRANE HILL,AL,35053.0,874899,CONSULTANTS-BUSINESS NEC,54161830.0,OTHER MANAGEMENT CONSULTING SERVICES,861107.0,FARM ORGANIZATIONS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,INTERNATIONAL ENTERPRISES INC,108 ALLEN ST,TALLADEGA,AL,35160.0,173106,ELECTRONIC CONTROL SYSTEMS,23821012.0,ELECTRICAL CONTR & OTHER WIRING INSTALLATION C...,874876.0,DEFENSE CONTRACTORS,999966.0,FEDERAL GOVERNMENT CONTRACTORS,NaN,NaN,NaN,NaN,NaN,NaN,633877998.0
3,JACKSON ROOFING & REMOLDLING,3324 HIDDEN BROOK CIRCLE,HEADLAND,AL,36345.0,874899,CONSULTANTS-BUSINESS NEC,54161830.0,OTHER MANAGEMENT CONSULTING SERVICES,176111.0,SIDING CONTRACTORS,176109.0,ROOFING CONTRACTORS,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AEROSTAR WORLD INC,239 AIRPORT DR,HEADLAND,AL,36345.0,874822,AVIATION CONSULTANTS,54169074.0,OTHER SCIENTIFIC & TECHNICAL CONSULTING SERVICES,458102.0,AIRCRAFT ENGINES-SERVICING & MAINTENANCE,458104.0,AIRCRAFT SERVICING & MAINTENANCE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,R & M CLEARWATER & AIR INC,1112 GENEVA HWY,ENTERPRISE,AL,36330.0,874870,WATER PURIFICATION CONSULTING SERVICES,54169046.0,OTHER SCIENTIFIC & TECHNICAL CONSULTING SERVICES,738984.0,WATER TREATMENT EQUIP SVC & SUPLS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,BOYKIN MANAGEMENT & CONSLTNG,733 MARVIN PARKER RD,OZARK,AL,36360.0,874899,CONSULTANTS-BUSINESS NEC,54161830.0,OTHER MANAGEMENT CONSULTING SERVICES,999966.0,FEDERAL GOVERNMENT CONTRACTORS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,VINLAND COMMUNICATIONS SVC,PO BOX 261,ARAB,AL,35016.0,874899,CONSULTANTS-BUSINESS NEC,54161830.0,OTHER MANAGEMENT CONSULTING SERVICES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,SANDERS AVIATION INC,20 ELLIS HAYNES DR,JASPER,AL,35503.0,874822,AVIATION CONSULTANTS,54169074.0,OTHER SCIENTIFIC & TECHNICAL CONSULTING SERVICES,999966.0,FEDERAL GOVERNMENT CONTRACTORS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,TENNESSEE RIVER MUSIC,553 RANDY OWEN DR NE,FORT PAYNE,AL,35967.0,874899,CONSULTANTS-BUSINESS NEC,54161830.0,OTHER MANAGEMENT CONSULTING SERVICES,75102.0,LIVESTOCK BREEDERS,599929.0,PET SUPPLIES & FOODS-RETAIL,519119.0,LIVESTOCK EQUIPMENT & SUPPLIES (WHLS),NaN,NaN,NaN,NaN,NaN


In [90]:
igroup_sic_8748_dict['MS']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,SOUTHERN INDUSTRIAL TECH,25 COUNTY ROAD 17,STRINGER,MS,39481.0,874836,INDUSTRIAL CONSULTANTS,54132012.0,LANDSCAPE ARCHITECTURAL SERVICES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,FURR & ASSOC,1040 BELT LINE DR NE,BROOKHAVEN,MS,39601.0,874823,COMMUNICATIONS CONSULTANTS,54161814.0,OTHER MANAGEMENT CONSULTING SERVICES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BEEHIVE HOMES OF PHILADELPHIA,708 COLUMBUS AVE,PHILADELPHIA,MS,39350.0,836105,RESIDENTIAL CARE HOMES,62331206.0,ASSISTED LIVING FACILITIES FOR THE ELDERLY,874859.0,ASSISTED LIVING FACILITY CONSULTANTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CHOCTAW SENIOR CITIZEN'S CTR,118 ELDERLY CENTER DR PEARL RI,PHILADELPHIA,MS,39350.0,836105,RESIDENTIAL CARE HOMES,62331206.0,ASSISTED LIVING FACILITIES FOR THE ELDERLY,874859.0,ASSISTED LIVING FACILITY CONSULTANTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,845636992.0


In [91]:
igroup_sic_8748_dict['IL']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,WAGNER CONSULTING-AGRICULTURE,709 OAK ST,LENA,IL,61048.0,874899,CONSULTANTS-BUSINESS NEC,54161830.0,OTHER MANAGEMENT CONSULTING SERVICES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ASSIST NATURAL PRODUCTS,218 EVERGREEN DR,LENA,IL,61048.0,399903,MANUFACTURERS,33999936.0,ALL OTHER MISCELLANEOUS MANUFACTURING,874820.0,AGRICULTURAL CONSULTANTS,21101.0,LIVESTOCK FEEDING,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
igroup_sic_8748_dict['IN']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,NEXT GENERATIONS DAIRY LLC,693 E STATE ROAD 218,BERNE,IN,46711.0,874849,DAIRY CONSULTANTS,54169044.0,OTHER SCIENTIFIC & TECHNICAL CONSULTING SERVICES,999966.0,FEDERAL GOVERNMENT CONTRACTORS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DAVITA VINCENNES HOME DIALYSIS,700 WILLOW ST # 102,VINCENNES,IN,47591.0,809203,DIALYSIS,62149202.0,KIDNEY DIALYSIS CENTERS,874802.0,TRAINING CONSULTANTS,829931.0,TRAINING PROGRAMS & SERVICES,801104.0,CLINICS,808201.0,HOME HEALTH SERVICE,NaN,NaN,886835255.0
2,MIKE'S FENCING,68021 STATE ROAD 19,NAPPANEE,IN,46550.0,738216,INSTALLATION/MONITORING-SECURITY SYSTEMS,56162117.0,SECURITY SYSTEMS SERVICES (EXCEPT LOCKSMITHS),179938.0,FENCE CONTRACTORS,154206.0,GATE & FENCE REPAIR,874813.0,SECURITY SYSTEMS CONSULTANTS,503902.0,FENCE POSTS & FITTINGS (WHLS),NaN,NaN,NaN


In [93]:
igroup_sic_8748_dict['OH']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,SECURITY SYSTEMS BUCKLAND,105 S MAIN ST,BUCKLAND,OH,45819.0,737415,INTERNET SERVICE,51821013.0,"DATA PROCESSING, HOSTING & RELATED SERVICES",874813.0,SECURITY SYSTEMS CONSULTANTS,481304.0,TELEPHONE COMPANIES,484101.0,TELEVISION-CABLE & CATV,NaN,NaN,NaN,NaN,NaN


In [94]:
igroup_sic_8748_dict['TN']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,CAPPS & CASTEELE ROOFING,1145 BRIARPATCH LAKE RD,PARIS,TN,38242.0,176109,ROOFING CONTRACTORS,23816014.0,ROOFING CONTRACTORS,176103.0,GUTTERS & DOWNSPOUTS,874809.0,ROOFING SERVICE CONSULTANTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [95]:
igroup_sic_8748_dict['AR']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,CTP ADVISORS,1458 PLAZA PL,SPRINGDALE,AR,72764.0,874899,CONSULTANTS-BUSINESS NEC,54161830.0,OTHER MANAGEMENT CONSULTING SERVICES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DALE CARNEGIE TRAINING,2837 AMERICAN ST # B1,SPRINGDALE,AR,72764.0,874802,TRAINING CONSULTANTS,54161212.0,HUMAN RESOURCES CONSULTING SERVICES,874802.0,TRAINING CONSULTANTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3977956.0
2,MORELIA MEXICAN REATSURANT,304 W SUNSET AVE,SPRINGDALE,AR,72764.0,874831,FOOD FACILITIES-CONSULTANTS,54161821.0,OTHER MANAGEMENT CONSULTING SERVICES,581208.0,RESTAURANTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
igroup_sic_8748_dict['NC']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,JOHNATHAN'S HOUSE,PO BOX 42,KIPLING,NC,27543.0,874825,EDUCATIONAL CONSULTANTS,61171012.0,EDUCATIONAL SUPPORT SERVICES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CAROLINA WATER TECHNOLOGY,22 NATIONAL GUARD ARMORY RD N,TAYLORSVILLE,NC,28681.0,874870,WATER PURIFICATION CONSULTING SERVICES,54169046.0,OTHER SCIENTIFIC & TECHNICAL CONSULTING SERVICES,549903.0,WATER COMPANIES-BOTTLED/BULK & ETC,738984.0,WATER TREATMENT EQUIP SVC & SUPLS,738984.0,WATER TREATMENT EQUIP SVC & SUPLS,NaN,NaN,NaN,NaN,NaN
2,CAROLINA WATER TECHNOLOGY,22 NATIONAL GUARD ARMORY RD N,TAYLORSVILLE,NC,28681.0,874870,WATER PURIFICATION CONSULTING SERVICES,54169046.0,OTHER SCIENTIFIC & TECHNICAL CONSULTING SERVICES,549903.0,WATER COMPANIES-BOTTLED/BULK & ETC,738984.0,WATER TREATMENT EQUIP SVC & SUPLS,738984.0,WATER TREATMENT EQUIP SVC & SUPLS,NaN,NaN,NaN,NaN,NaN
3,CAROLINA WATER TECHNOLOGY,22 NATIONAL GUARD ARMORY RD N,TAYLORSVILLE,NC,28681.0,874870,WATER PURIFICATION CONSULTING SERVICES,54169046.0,OTHER SCIENTIFIC & TECHNICAL CONSULTING SERVICES,549903.0,WATER COMPANIES-BOTTLED/BULK & ETC,738984.0,WATER TREATMENT EQUIP SVC & SUPLS,738984.0,WATER TREATMENT EQUIP SVC & SUPLS,NaN,NaN,NaN,NaN,NaN
4,M & L FIRE PROTECTION,6420 GEORGE HILDEBRAN SCH RD,HICKORY,NC,28602.0,506315,FIRE PROTECTION EQUIPMENT & SUPLS (WHLS),42361033.0,ELECTRICAL APPARATUS/WIRING SUPLS/REL EQUIP WH...,874832.0,FIRE PROTECTION CONSULTANTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,BLUE RIDGE HOME CONSLNTS INC,144 MARSHALL RD,NEBO,NC,28761.0,874899,CONSULTANTS-BUSINESS NEC,54161830.0,OTHER MANAGEMENT CONSULTING SERVICES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,SOUTHEASTERN LOSS MANAGEMENT,207 N OAKLAND ST,DALLAS,NC,28034.0,874861,EMPLOYMENT SCREENING SERVICES,54161828.0,OTHER MANAGEMENT CONSULTING SERVICES,738110.0,DETECTIVES-PRIVATE,738104.0,DETECTIVE AGENCIES,738108.0,INVESTIGATORS,NaN,NaN,NaN,NaN,NaN
7,TUCKER & TUCKER CONSULTING LLC,2321 MADEIRA CIR,WAXHAW,NC,28173.0,874899,CONSULTANTS-BUSINESS NEC,54161830.0,OTHER MANAGEMENT CONSULTING SERVICES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,714 CONSULTING LLC,501 E MAIN AVE,TAYLORSVILLE,NC,28681.0,874899,CONSULTANTS-BUSINESS NEC,54161830.0,OTHER MANAGEMENT CONSULTING SERVICES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,CATAWBA COUNTY ABC BOARD,42 CAROLINA LN,TAYLORSVILLE,NC,28681.0,592102,LIQUORS-RETAIL,44531004.0,"BEER, WINE & LIQUOR STORES",874864.0,GOVERNMENT-RELATIONS CONSULTANTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,883273070.0


## SIC Code 5431 Exploration: Farm Markets

In [98]:
# To see how many matches exist within the dataframe consisting of SIC Codes containing 0762
df_sic_5431 = pd.read_csv("../data/code_5431.csv")
  
igroup_sic_5431_dict, ctrglow_5431_dict = infogroup_counterglow_dict(df_sic_5431, df_counterglow_poultry, states, RADIUS_KM)


IA: Total number of matches: 2
AL: Total number of matches: 1
IN: Total number of matches: 1
OH: Total number of matches: 1
AR: Total number of matches: 5
NC: Total number of matches: 7
GA: Total number of matches: 1


In [99]:
igroup_sic_5431_dict = keep_certain_columns(igroup_sic_5431_dict)

#### Exploration

In [100]:
igroup_sic_5431_dict['IA']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,FARMERS FRESH POULTRY,5195 FARMERS AVE SW,KALONA,IA,52247,25401,POULTRY FARMS,11234001.0,POULTRY HATCHERIES,543102.0,FARM MARKETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,FLINT RIDGE ORGANIC PRODUCE,1896 110TH ST,KALONA,IA,52247,543101,FRUITS & VEGETABLES & PRODUCE-RETAIL,44523003.0,FRUIT & VEGETABLE MARKETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
igroup_sic_5431_dict['AL']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,MID CITY FARM MARKET,14871 S US HIGHWAY 231,MIDLAND CITY,AL,36350,543101,FRUITS & VEGETABLES & PRODUCE-RETAIL,44523003.0,FRUIT & VEGETABLE MARKETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
igroup_sic_5431_dict['IN']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,SHIREMAN'S FARM MARKET,2081 HIGHWAY 135 NEW,CORYDON,IN,47112,543101,FRUITS & VEGETABLES & PRODUCE-RETAIL,44523003.0,FRUIT & VEGETABLE MARKETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [103]:
igroup_sic_5431_dict['OH']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,PICKIN' PATCH,9460 RAVENNA AVE,LOUISVILLE,OH,44641,543101,FRUITS & VEGETABLES & PRODUCE-RETAIL,44523003.0,FRUIT & VEGETABLE MARKETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
igroup_sic_5431_dict['AR']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,DEL,1229 W SUNSET AVE,SPRINGDALE,AR,72764,543101,FRUITS & VEGETABLES & PRODUCE-RETAIL,44523003.0,FRUIT & VEGETABLE MARKETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LOS ALAMOS,503 HOLCOMB ST,SPRINGDALE,AR,72764,541105,GROCERS-RETAIL,44511003.0,SUPERMARKETS/OTHER GROCERY (EXC CONVENIENCE) STRS,543101.0,FRUITS & VEGETABLES & PRODUCE-RETAIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,H C SCHMIEDING PRODUCE,PO BOX 369,SPRINGDALE,AR,72765,514803,PRODUCE-BROKERS (WHLS),42512057.0,WHOLESALE TRADE AGENTS & BROKERS,999966.0,FEDERAL GOVERNMENT CONTRACTORS,514801.0,FRUITS & VEGETABLES-WHOLESALE,543101.0,FRUITS & VEGETABLES & PRODUCE-RETAIL,NaN,NaN,NaN,NaN,NaN
3,TWIN CITY PRODUCE,1100 71 PLAZA CT # 11,SPRINGDALE,AR,72764,514801,FRUITS & VEGETABLES-WHOLESALE,42448009.0,FRESH FRUIT & VEGETABLE MERCHANT WHOLESALERS,543101.0,FRUITS & VEGETABLES & PRODUCE-RETAIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DEL REY PRODUCE,1229 W SUNSET AVE,SPRINGDALE,AR,72764,543101,FRUITS & VEGETABLES & PRODUCE-RETAIL,44523003.0,FRUIT & VEGETABLE MARKETS,514803.0,PRODUCE-BROKERS (WHLS),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [105]:
igroup_sic_5431_dict['NC']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,TRADING POST,4500 US HIGHWAY 64 90 W,TAYLORSVILLE,NC,28681,543101,FRUITS & VEGETABLES & PRODUCE-RETAIL,44523003.0,FRUIT & VEGETABLE MARKETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TRADING POST,4500 US HIGHWAY 64 90 W,TAYLORSVILLE,NC,28681,543101,FRUITS & VEGETABLES & PRODUCE-RETAIL,44523003.0,FRUIT & VEGETABLE MARKETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TRADING POST,4500 US HIGHWAY 64 90 W,TAYLORSVILLE,NC,28681,543101,FRUITS & VEGETABLES & PRODUCE-RETAIL,44523003.0,FRUIT & VEGETABLE MARKETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TRADING POST,4500 US HIGHWAY 64 90 W,TAYLORSVILLE,NC,28681,543101,FRUITS & VEGETABLES & PRODUCE-RETAIL,44523003.0,FRUIT & VEGETABLE MARKETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DEBS PRODUCE MKT & BAIT SHOP,2021 HARMONY GROVE RD,NEBO,NC,28761,543101,FRUITS & VEGETABLES & PRODUCE-RETAIL,44523003.0,FRUIT & VEGETABLE MARKETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,PIEDMONT PRODUCE,4212 E HIGHWAY 218,MONROE,NC,28110,543101,FRUITS & VEGETABLES & PRODUCE-RETAIL,44523003.0,FRUIT & VEGETABLE MARKETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,JAMESTOWN FLEA MARKET,709 JAMESTOWN RD,MORGANTON,NC,28655,593218,FLEA MARKETS,45331017.0,USED MERCHANDISE STORES,543102.0,FARM MARKETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [106]:
igroup_sic_5431_dict['GA']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,KAUFFMAN STRAWBERRIES,1305 MENNONITE CHURCH RD,MONTEZUMA,GA,31063,543101,FRUITS & VEGETABLES & PRODUCE-RETAIL,44523003.0,FRUIT & VEGETABLE MARKETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## SIC Code 8611 Exploration: Farm Organizations

In [107]:
# To see how many matches exist within the dataframe consisting of SIC Codes containing 0762
df_sic_8611 = pd.read_csv("../data/code_8611.csv")
  
igroup_sic_8611_dict, ctrglow_8611_dict = infogroup_counterglow_dict(df_sic_8611, df_counterglow_poultry, states, RADIUS_KM)


IA: Total number of matches: 3
MO: Total number of matches: 1
AL: Total number of matches: 8
MS: Total number of matches: 6
IL: Total number of matches: 1
IN: Total number of matches: 5
TN: Total number of matches: 1
AR: Total number of matches: 3
NC: Total number of matches: 11
GA: Total number of matches: 2


In [108]:
igroup_sic_8611_dict = keep_certain_columns(igroup_sic_8611_dict)

#### Exploration

In [109]:
igroup_sic_8611_dict['IA']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,BEEF PROMOTIONS OF IOWA INC,2430 240TH ST,DALLAS CENTER,IA,50063,861102,ASSOCIATIONS,81391002.0,BUSINESS ASSOCIATIONS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DAIRY HERD IMPROVEMENT ASSOC,3941 DOVE AVE,SIOUX CENTER,IA,51250,839998,NON-PROFIT ORGANIZATIONS,81331908.0,OTHER SOCIAL ADVOCACY ORGANIZATIONS,861101.0,BUSINESS & TRADE ORGANIZATIONS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SUNRISE HOMES CO-OP OF HOSPERS,106 COUNTY RD N,HOSPERS,IA,51238,861102,ASSOCIATIONS,81391002.0,BUSINESS ASSOCIATIONS,653107.0,COOPERATIVES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [110]:
igroup_sic_8611_dict['MO']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,DAVIDIAN SDA ASSN,NaN,EXETER,MO,65647,861102,ASSOCIATIONS,81391002.0,BUSINESS ASSOCIATIONS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [111]:
igroup_sic_8611_dict['AL']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,ARAB CHAMBER OF COMMERCE,1157 N MAIN ST,ARAB,AL,35016,861104,CHAMBERS OF COMMERCE,81391005.0,BUSINESS ASSOCIATIONS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4612800.0
1,RANDY DAVIS FARM,122 COUNTY ROAD 856,CRANE HILL,AL,35053,874899,CONSULTANTS-BUSINESS NEC,54161830.0,OTHER MANAGEMENT CONSULTING SERVICES,861107.0,FARM ORGANIZATIONS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CLAY COUNTY BAPTIST CTR,60084 HIGHWAY 49,LINEVILLE,AL,36266,861102,ASSOCIATIONS,81391002.0,BUSINESS ASSOCIATIONS,866110.0,RELIGIOUS ORGANIZATIONS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CLAY COUNTY BAPTIST CTR,60084 HIGHWAY 49,LINEVILLE,AL,36266,861102,ASSOCIATIONS,81391002.0,BUSINESS ASSOCIATIONS,866110.0,RELIGIOUS ORGANIZATIONS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CLAY COUNTY CHAMBER-COMMERCE,88855 HIGHWAY 9,LINEVILLE,AL,36266,861104,CHAMBERS OF COMMERCE,81391005.0,BUSINESS ASSOCIATIONS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4612800.0
5,CLAY COUNTY CHAMBER-COMMERCE,88855 HIGHWAY 9,LINEVILLE,AL,36266,861104,CHAMBERS OF COMMERCE,81391005.0,BUSINESS ASSOCIATIONS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4612800.0
6,HEARTS,21517 MAIN ST,RANBURNE,AL,36273,839998,NON-PROFIT ORGANIZATIONS,81331908.0,OTHER SOCIAL ADVOCACY ORGANIZATIONS,861102.0,ASSOCIATIONS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,ELBA HOUSING AUTHORITY,1207 CLAXTON AVE N,ELBA,AL,36323,653132,HOUSING AUTHORITIES,53131106.0,RESIDENTIAL PROPERTY MANAGERS,912104.0,GOVERNMENT OFFICES-CITY/VILLAGE & TWP,999966.0,FEDERAL GOVERNMENT CONTRACTORS,861102.0,ASSOCIATIONS,653132.0,HOUSING AUTHORITIES,NaN,NaN,NaN


In [112]:
igroup_sic_8611_dict['MS']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,MSCI,NaN,PHILADELPHIA,MS,39350,861101,BUSINESS & TRADE ORGANIZATIONS,81391004.0,BUSINESS ASSOCIATIONS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,FREENY WATER ASSN,1411 HIGHWAY 35 S,CARTHAGE,MS,39051,861102,ASSOCIATIONS,81391002.0,BUSINESS ASSOCIATIONS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MAGEE BENEVOLENT FOUNDATION,300 3RD AVE SE,MAGEE,MS,39111,873303,FOUNDATION-EDUC PHILANTHROPIC RESEARCH,54172006.0,RESEARCH & DEVMNT-SOCIAL SCIENCES & HUMANITIES,999966.0,FEDERAL GOVERNMENT CONTRACTORS,861102.0,ASSOCIATIONS,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GIRL SCOUTS-GULF PINES COUNCIL,11 CAMP ITA KANA RD,WIGGINS,MS,39577,864123,GIRL SCOUTS,81341025.0,CIVIL & SOCIAL ORGANIZATIONS,703203.0,CAMPS,832222.0,YOUTH ORGANIZATIONS & CENTERS,861102.0,ASSOCIATIONS,864101.0,FRATERNAL ORGANIZATIONS,NaN,NaN,6972012.0
4,PHILADELPHIA-NESHOBA CHAMBER,256 W BEACON ST,PHILADELPHIA,MS,39350,861101,BUSINESS & TRADE ORGANIZATIONS,81391004.0,BUSINESS ASSOCIATIONS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NESHOBA COUNTY FARM BUREAU,704 E MAIN ST,PHILADELPHIA,MS,39350,861107,FARM ORGANIZATIONS,81391006.0,BUSINESS ASSOCIATIONS,641112.0,INSURANCE,839998.0,NON-PROFIT ORGANIZATIONS,NaN,NaN,NaN,NaN,NaN,NaN,406588095.0


In [113]:
igroup_sic_8611_dict['IL']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,AMERICAN LEGION HOME,316 W MAIN ST,LENA,IL,61048,864102,VETERANS' & MILITARY ORGANIZATIONS,81341003.0,CIVIL & SOCIAL ORGANIZATIONS,861102.0,ASSOCIATIONS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,496227620.0


In [114]:
igroup_sic_8611_dict['IN']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,MENTONE CHAMBER OF COMMERCE,105 E MAIN ST,MENTONE,IN,46539,861104,CHAMBERS OF COMMERCE,81391005.0,BUSINESS ASSOCIATIONS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4612800.0
1,INDIANA INSTITUTE OF TECH INC,1223 W WATER ST,BERNE,IN,46711,861102,ASSOCIATIONS,81391002.0,BUSINESS ASSOCIATIONS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,VINCENNES AREA BUILDERS ASSN,1421 MENTOR ST,VINCENNES,IN,47591,152103,GENERAL CONTRACTORS,23611505.0,NEW SINGLE-FAMILY HSNG CONSTR (EXC FOR-SALE BL...,861102.0,ASSOCIATIONS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,KNOX COUNTY CHAMBER-COMMERCE,316 MAIN ST,VINCENNES,IN,47591,861104,CHAMBERS OF COMMERCE,81391005.0,BUSINESS ASSOCIATIONS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4612800.0
4,KNOX COUNTY CMNTY FOUNDATION,20 N 3RD ST # 301,VINCENNES,IN,47591,861102,ASSOCIATIONS,81391002.0,BUSINESS ASSOCIATIONS,864110.0,COMMUNITY ORGANIZATIONS,839998.0,NON-PROFIT ORGANIZATIONS,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
igroup_sic_8611_dict['TN']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,POPULAR ROTORCRAFT ASSN INC,3583 LEAD MINE VALLEY RD SW,CLEVELAND,TN,37311,861102,ASSOCIATIONS,81391002.0,BUSINESS ASSOCIATIONS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [116]:
igroup_sic_8611_dict['AR']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,MERCHAB INSTITUTE,609 W MAPLE AVE,SPRINGDALE,AR,72764,861102,ASSOCIATIONS,81391002.0,BUSINESS ASSOCIATIONS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NORTHWEST ARK HM BLDRS ASSN,2022 LONG ST,SPRINGDALE,AR,72764,861102,ASSOCIATIONS,81391002.0,BUSINESS ASSOCIATIONS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SPRINGDALE CHAMBER OF COMMERCE,1200 S THOMPSON ST,SPRINGDALE,AR,72764,861104,CHAMBERS OF COMMERCE,81391005.0,BUSINESS ASSOCIATIONS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [117]:
igroup_sic_8611_dict['NC']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,SPENCER MOUNTAIN VLG HOA POOL,1620 EASTWAY DR,DALLAS,NC,28034,861102,ASSOCIATIONS,81391002.0,BUSINESS ASSOCIATIONS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ALEXANDER BAPTIST ASSN,215 5TH AVE SW,TAYLORSVILLE,NC,28681,861102,ASSOCIATIONS,81391002.0,BUSINESS ASSOCIATIONS,866110.0,RELIGIOUS ORGANIZATIONS,866107.0,CHURCHES,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ALEXANDER COUNTY ANCESTOR INC,205 MAIN AVENUE DR,TAYLORSVILLE,NC,28681,861102,ASSOCIATIONS,81391002.0,BUSINESS ASSOCIATIONS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ALEXANDER COUNTY ANCESTOR INC,205 MAIN AVENUE DR,TAYLORSVILLE,NC,28681,861102,ASSOCIATIONS,81391002.0,BUSINESS ASSOCIATIONS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ALEXANDER COUNTY ANCESTOR INC,205 MAIN AVENUE DR,TAYLORSVILLE,NC,28681,861102,ASSOCIATIONS,81391002.0,BUSINESS ASSOCIATIONS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,HOYLE FAMILY HOMESTEAD,1214 DALLAS STANLEY HWY,DALLAS,NC,28034,861102,ASSOCIATIONS,81391002.0,BUSINESS ASSOCIATIONS,841201.0,MUSEUMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,QUEEN'S CUP STEEPLECHASE,6103 WAXHAW HWY,MINERAL SPRINGS,NC,28108,861102,ASSOCIATIONS,81391002.0,BUSINESS ASSOCIATIONS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,QUEEN'S CUP STEEPLECHASE,6103 WAXHAW HWY,MINERAL SPRINGS,NC,28108,861102,ASSOCIATIONS,81391002.0,BUSINESS ASSOCIATIONS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,RESOURCE INSTITUTE INC,4281 BILL EPLEY AVE,MORGANTON,NC,28655,861102,ASSOCIATIONS,81391002.0,BUSINESS ASSOCIATIONS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,RESOURCE INSTITUTE INC,4281 BILL EPLEY AVE,MORGANTON,NC,28655,861102,ASSOCIATIONS,81391002.0,BUSINESS ASSOCIATIONS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [118]:
igroup_sic_8611_dict['GA']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,EASTER SEALS,1212 JENKINS DR,VIENNA,GA,31092,832218,SOCIAL SERVICE & WELFARE ORGANIZATIONS,62419012.0,OTHER INDIVIDUAL & FAMILY SERVICES,861102.0,ASSOCIATIONS,703203.0,CAMPS,804918.0,PHYSICAL THERAPISTS,808201.0,HOME HEALTH SERVICE,832215.0,MARRIAGE & FAMILY COUNSELORS,151644457.0
1,EASTER SEALS,1301 E UNION ST,VIENNA,GA,31092,832218,SOCIAL SERVICE & WELFARE ORGANIZATIONS,62419012.0,OTHER INDIVIDUAL & FAMILY SERVICES,839919.0,CHARITABLE INSTITUTIONS,861102.0,ASSOCIATIONS,703203.0,CAMPS,804918.0,PHYSICAL THERAPISTS,808201.0,HOME HEALTH SERVICE,151644457.0


## SIC Code 0723 Exploration: Farm Produce

In [119]:
# To see how many matches exist within the dataframe consisting of SIC Codes containing 0762
df_sic_0723 = pd.read_csv("../data/code_0723.csv")
  
igroup_sic_0723_dict, ctrglow_0723_dict = infogroup_counterglow_dict(df_sic_0723, df_counterglow_poultry, states, RADIUS_KM)


NC: Total number of matches: 3


In [120]:
igroup_sic_0723_dict = keep_certain_columns(igroup_sic_0723_dict)

#### Explore

In [121]:
igroup_sic_0723_dict['NC']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,FARMER'S DAUGHTER,2150 FRIENDSHIP CHURCH RD,TAYLORSVILLE,NC,28681,72311,FARM PRODUCE,11511410.0,OTHER POST-HARVEST CROP ACTIVITIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,FARMER'S DAUGHTER,2150 FRIENDSHIP CHURCH RD,TAYLORSVILLE,NC,28681,72311,FARM PRODUCE,11511410.0,OTHER POST-HARVEST CROP ACTIVITIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,FARMER'S DAUGHTER,2150 FRIENDSHIP CHURCH RD,TAYLORSVILLE,NC,28681,72311,FARM PRODUCE,11511410.0,OTHER POST-HARVEST CROP ACTIVITIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## SIC Code 0191 Exploration: Farm Products

In [123]:
# To see how many matches exist within the dataframe consisting of SIC Codes containing 0762
df_sic_0191 = pd.read_csv("../data/code_0191.csv")
  
igroup_sic_0191_dict, ctrglow_0191_dict = infogroup_counterglow_dict(df_sic_0191, df_counterglow_poultry, states, RADIUS_KM)


IA: Total number of matches: 46
MO: Total number of matches: 25
AL: Total number of matches: 55
LA: Total number of matches: 1
MS: Total number of matches: 23
IL: Total number of matches: 1
IN: Total number of matches: 19
OH: Total number of matches: 6
KY: Total number of matches: 2
TN: Total number of matches: 1
AR: Total number of matches: 2
NC: Total number of matches: 48
SC: Total number of matches: 4
GA: Total number of matches: 5


In [125]:
igroup_sic_0191_dict = keep_certain_columns(igroup_sic_0191_dict)

#### Explore

In [126]:
igroup_sic_0191_dict['IA']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,KDD INC,4755 20TH ST,GRINNELL,IA,50112,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,999966.0,FEDERAL GOVERNMENT CONTRACTORS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CENTRUM VALLEY FARMS,2975 WASHINGTON AVE,DOWS,IA,50071,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CENTRUM VALLEY FARMS,2975 WASHINGTON AVE,DOWS,IA,50071,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AXMEAR AG INC,1065 OLD 6 RD,MALCOM,IA,50157,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CENTURN VALLEY FARMS,2688 270TH ST,GALT,IA,50101,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,HINDMAN PORK,15270 280 ST,RADCLIFFE,IA,50230,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,CENTRUM VALLEY FARMS,2674 HIGHWAY 69,GALT,IA,50101,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,514403.0,EGGS (WHLS),541105.0,GROCERS-RETAIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,CENTRUM VALLEY FARMS,2674 HIGHWAY 69,GALT,IA,50101,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,514403.0,EGGS (WHLS),541105.0,GROCERS-RETAIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,AX BROTHERS AG LLC,1065 OLD 6 RD,MALCOM,IA,50157,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,DAVID HOAKISON FARMS,1089 JOSHUA AVE,CRESTON,IA,50801,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [127]:
igroup_sic_0191_dict['MO']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,TRI-STATE FARM & FEED,307 S MAIN ST,SOUTH WEST CITY,MO,64863,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,539905.0,FARMERS CO-OP RETAIL STORES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TRI-STATE FARM & FEED,307 S MAIN ST,SOUTH WEST CITY,MO,64863,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,539905.0,FARMERS CO-OP RETAIL STORES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KELLEY FARMS,220 BLUE CLAY RIDGE RD,ROCKY COMFORT,MO,64861,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,JOHNSON COUNTY EGG FARM,1275 SE Y HWY,KNOB NOSTER,MO,65336,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,514403.0,EGGS (WHLS),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,OUTBACK HENHOUSE,30947 LEE RD,SEDALIA,MO,65301,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,JOHNSON COUNTY EGG FARM,1285 SE Y HWY,KNOB NOSTER,MO,65336,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,514403.0,EGGS (WHLS),549914.0,EGGS-RETAIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,DANNY MAZELIN FARM,33403 ALMOND RD,COLE CAMP,MO,65325,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,DANNY MAZELIN FARM,33403 ALMOND RD,COLE CAMP,MO,65325,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,BURNS FARMS,4367 STATE HIGHWAY HH,PURDY,MO,65734,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,BURNS FARMS,4367 STATE HIGHWAY HH,PURDY,MO,65734,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [128]:
igroup_sic_0191_dict['AL']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,HAYNES FARM INC,5084 COUNTY HIGHWAY 49,BLOUNTSVILLE,AL,35031,421304,TRUCKING,48423013.0,SPECIALIZED FREIGHT (EXC USED GDS) TRCKNG LNG-...,19101.0,FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TRIPP F L,100 EMBER DR,ARAB,AL,35016,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NUNLEY FARMS LLC,12527 HIGHWAY 23,VINA,AL,35593,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,WENDELL GIBBS FARMS,2118 COUNTY ROAD 23,RANBURNE,AL,36273,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MOORE FARMS & FRIENDS LLC,239 COUNTY ROAD 561,WOODLAND,AL,36280,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,C J FARMS,486 COUNTY ROAD 95,RANBURNE,AL,36273,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,C J FARMS,486 COUNTY ROAD 95,RANBURNE,AL,36273,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,HARTZOG FARMS,304 WALDEN POND RD,HEADLAND,AL,36345,21202,BEEF CATTLE-EXCEPT FEEDLOTS,11211101.0,BEEF CATTLE RANCHING & FARMING,19198.0,GENERAL FARMS-PRIMARILY CROP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,GRANBERRY FARM,5955 COUNTY ROAD 55,COLUMBIA,AL,36319,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,WILLIAM MOORE FARMS,239 COUNTY ROAD 561,WOODLAND,AL,36280,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [129]:
igroup_sic_0191_dict['LA']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,BROOK FARMS ENTERPRISES LLC,120 GAME RESERVE RD,MARION,LA,71260,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [130]:
igroup_sic_0191_dict['MS']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,TRIANGLE B FARMS INC,2116 BEELINE RD,DECATUR,MS,39327,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,999966.0,FEDERAL GOVERNMENT CONTRACTORS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TCL POULTRY FARM,83 COURTNEY LN,BASSFIELD,MS,39421,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ROWLEY FARMS INC,313 ROWLEY RD,FOXWORTH,MS,39483,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,999966.0,FEDERAL GOVERNMENT CONTRACTORS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,COTTON BRANCH PLANTATION,11670 HIGHWAY 98,SMITHDALE,MS,39664,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,823106.0,LIBRARIES-PUBLIC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,B B FARMS,667 LOWER BROOKHAVEN RD,MONTICELLO,MS,39654,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,B B FARMS,667 LOWER BROOKHAVEN RD,MONTICELLO,MS,39654,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,MARTIN FARMS LLC,12302 HIGHWAY 98,SMITHDALE,MS,39664,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,INDIAN CREEK FARMS LLC,167 INDIAN CREEK LN,KOKOMO,MS,39643,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,WOOTEN FARMS,300 HIGHWAY 487 E,CARTHAGE,MS,39051,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,HOMOCHITTO POULTRY FARM,3118 CASEYVILLE RD NW,WESSON,MS,39191,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [131]:
igroup_sic_0191_dict['IL']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,HI-GRADE EGG,317 E 360 NORTH RD # 2,LODA,IL,60948,201501,POULTRY PROCESSING PLANTS (MFRS),31161501.0,POULTRY PROCESSING,19101.0,FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [132]:
igroup_sic_0191_dict['IN']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,MANWARING FARMS INC,4382 S 1000 W,MENTONE,IN,46539,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LA DON FARMS,13764 COUNTY ROAD 28,MIDDLEBURY,IN,46540,24103,DAIRY FARMS,11212002.0,DAIRY CATTLE & MILK PRODUCTION,19101.0,FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LA DON FARMS,13764 COUNTY ROAD 28,MIDDLEBURY,IN,46540,24103,DAIRY FARMS,11212002.0,DAIRY CATTLE & MILK PRODUCTION,19101.0,FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,F & A HOG FARM LLC,2150 W 150 N,LAGRANGE,IN,46761,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ISCH FARMS INC,1977 S 400 W-1,BLUFFTON,IN,46714,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,S & G POULTRY FARM,2496 S 400 W,MONROE,IN,46772,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,25401.0,POULTRY FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,CRANDALL FARMS,3319 E COUNTY ROAD 500 N,ORLEANS,IN,47452,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,999966.0,FEDERAL GOVERNMENT CONTRACTORS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,J & K TURKEY FARM LLC,4329 E 225 S,WASHINGTON,IN,47501,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,KNEPP FARMS INC,5177 E 200 S,MONTGOMERY,IN,47558,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,VEALCREEK TURKEYS,5529 E 200 S,MONTGOMERY,IN,47558,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,25301.0,TURKEY FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [133]:
igroup_sic_0191_dict['OH']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,HERTZFELD POULTRY FARMS,15799 MILTON RD,GRAND RAPIDS,OH,43522,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,25401.0,POULTRY FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ROYER FARMS INC,8101 STATE ST NE,LOUISVILLE,OH,44641,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,999966.0,FEDERAL GOVERNMENT CONTRACTORS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MUHLENKAMP FARMS,5924 STATE ROUTE 49,FORT RECOVERY,OH,45846,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HOMAN FARMS,6671 ASHMAN RD,MARIA STEIN,OH,45860,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,13901.0,AGRICULTURAL PRODUCTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PHILLIPS BROTHERS FARMS LLC,11721 BUCKLAND HOLDEN RD,WAPAKONETA,OH,45895,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,999966.0,FEDERAL GOVERNMENT CONTRACTORS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,M J R FARMS LLC,6492 HUWER RD,MARIA STEIN,OH,45860,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [134]:
igroup_sic_0191_dict['KY']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,BUCK CREEK FARM LLC,750 BARRETT HILL RD,LIVERMORE,KY,42352,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,999966.0,FEDERAL GOVERNMENT CONTRACTORS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,OLD OAK FARMS LLC,3390 US HIGHWAY 431 N,LIVERMORE,KY,42352,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [135]:
igroup_sic_0191_dict['TN']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,SUNSET RIDGE CATTLE CO,400 BRODIO RD,DEER LODGE,TN,37726,75102,LIVESTOCK BREEDERS,11521010.0,SUPPORT ACTIVITIES FOR ANIMAL PRODUCTION,19101.0,FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [136]:
igroup_sic_0191_dict['AR']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,U-PICK IT FARM & GARDEN CORP,17369 WYOLA RD,WINSLOW,AR,72959,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,UNDERWOOD FARMS,10306 LONNIE LN,DANVILLE,AR,72833,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [138]:
igroup_sic_0191_dict['NC']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,LEONARD LAKES FARMS,329 LEONARD RD,LINCOLNTON,NC,28092,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BROOKLANDWOOD FARMS LLC,3830 S POTTER RD,MONROE,NC,28112,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CONYERS' FARMS,8520 COLLIERS CHAPEL CHURCH RD,LINDEN,NC,28356,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ROGERS MOUNTAINVIEW FARMS INC,407 E MAIN AVE,TAYLORSVILLE,NC,28681,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,R & J FARMS LLC,2989 BLACK OAK RIDGE RD,TAYLORSVILLE,NC,28681,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,R & J FARMS LLC,2989 BLACK OAK RIDGE RD,TAYLORSVILLE,NC,28681,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,R & J FARMS LLC,2989 BLACK OAK RIDGE RD,TAYLORSVILLE,NC,28681,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,HIGHLAND MOUNTAIN VIEW FARMS,450 PINE COVE RD,TAYLORSVILLE,NC,28681,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,TAYLOR FARM LLC,6291 OLD LAUREL RD,CONNELLY SPRINGS,NC,28612,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,JUNE BUG FARM,2631 JUNE BUG RD,VALE,NC,28168,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [139]:
igroup_sic_0191_dict['SC']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,MUNGO BROTHERS FARMS LLP,3527 ROCKY RIVER RD,HEATH SPRINGS,SC,29058,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LLOYD FARM,PO BOX 1,REMBERT,SC,29128,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,WILSON RIDGE FARM LLC,5116 WYLIES MILL RD,EDGEMOOR,SC,29712,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BELGER FARMS & CONSTR INC,2044 BOYKIN RD,REMBERT,SC,29128,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [140]:
igroup_sic_0191_dict['GA']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,HOUSE OF RAEFORD FARMS INC,55 COOPER RD,COMER,GA,30629,201501,POULTRY PROCESSING PLANTS (MFRS),31161501.0,POULTRY PROCESSING,19101.0,FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,513523.0
1,SAWTOOTH FARMS,9350 HWY 27 E,VIENNA,GA,31092,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,999966.0,FEDERAL GOVERNMENT CONTRACTORS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MINOR FARMS,3189 MENNONITE CHURCH RD,MONTEZUMA,GA,31063,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LANIER SIDNEY FARMS,3665 GA HIGHWAY 26 E,MONTEZUMA,GA,31063,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,999966.0,FEDERAL GOVERNMENT CONTRACTORS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TANNER FARMS,184 ROW BOAT RD,ALMA,GA,31510,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## SIC Code 254 Exploration: Poultry Farms

In [141]:
# To see how many matches exist within the dataframe consisting of SIC Codes containing 0762
df_sic_254 = pd.read_csv("../data/code_254.csv")
  
igroup_sic_254_dict, ctrglow_254_dict = infogroup_counterglow_dict(df_sic_254, df_counterglow_poultry, states, RADIUS_KM)


IA: Total number of matches: 8
MO: Total number of matches: 5
AL: Total number of matches: 9
LA: Total number of matches: 1
MS: Total number of matches: 6
IN: Total number of matches: 6
OH: Total number of matches: 2
AR: Total number of matches: 2
NC: Total number of matches: 3
GA: Total number of matches: 3


In [142]:
igroup_sic_254_dict = keep_certain_columns(igroup_sic_254_dict)

#### Explore

In [143]:
igroup_sic_254_dict['IA']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,VALO BIOMEDIA NORTH AMERICA,21459 OLD HIGHWAY 6,ADEL,IA,50003,283402,POULTRY MEDICINES & VACCINES (MFRS),32541208.0,PHARMACEUTICAL PREPARATION MANUFACTURING,25401.0,POULTRY FARMS,999966.0,FEDERAL GOVERNMENT CONTRACTORS,545101.0,DAIRY PRODUCTS-RETAIL,514404.0,POULTRY-WHOLESALE,NaN,NaN,843668294.0
1,VALO BIOMEDIA NORTH AMERICA,16332 OLD HIGHWAY 6,REDFIELD,IA,50233,283402,POULTRY MEDICINES & VACCINES (MFRS),32541208.0,PHARMACEUTICAL PREPARATION MANUFACTURING,873104.0,LABORATORIES,25401.0,POULTRY FARMS,NaN,NaN,NaN,NaN,NaN,NaN,843668294.0
2,CALEKO FARM INC,3819 330TH ST,BOYDEN,IA,51234,25401,POULTRY FARMS,11234001.0,POULTRY HATCHERIES,19101.0,FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,REMBRANDT ENTERPRISES,4252 80 AVE,LINN GROVE,IA,51033,25401,POULTRY FARMS,11234001.0,POULTRY HATCHERIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,REMBRANDT ENTERPRISES,4252 80 AVE,LINN GROVE,IA,51033,25401,POULTRY FARMS,11234001.0,POULTRY HATCHERIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,IOWA CAGE FREE LLP,1035 200TH ST,GOLDFIELD,IA,50542,25401,POULTRY FARMS,11234001.0,POULTRY HATCHERIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,FARMERS FRESH POULTRY,5195 FARMERS AVE SW,KALONA,IA,52247,25401,POULTRY FARMS,11234001.0,POULTRY HATCHERIES,543102.0,FARM MARKETS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,RIVERLAND POULTRY,2015 HIGHWAY 22,KALONA,IA,52247,25401,POULTRY FARMS,11234001.0,POULTRY HATCHERIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [144]:
igroup_sic_254_dict['MO']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,LEUSCHEN POULTRY OPERATIONS,1175 DAIRY CIR,CRANE,MO,65633,514402,POULTRY SERVICES (WHLS),42444002.0,POULTRY & POULTRY PRODUCT MERCHANT WHOLESALERS,25401.0,POULTRY FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,RIVERAS FARMS LLC,1316 FARM ROAD 2155,EXETER,MO,65647,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,25401.0,POULTRY FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SELECT GENETICS,18968 LAWRENCE 2130,MT VERNON,MO,65712,25401,POULTRY FARMS,11234001.0,POULTRY HATCHERIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TYSON FOODS INC,19597 STATE HWY E,BLOOMFIELD,MO,63825,25401,POULTRY FARMS,11234001.0,POULTRY HATCHERIES,514404.0,POULTRY-WHOLESALE,201501.0,POULTRY PROCESSING PLANTS (MFRS),201104.0,MEAT PRODUCTS (MFRS),204803.0,FEED-MANUFACTURERS,207702.0,RENDERING COMPANIES (MFRS),7537913.0
4,DICK MCKUIN FARMS,2320 STATE HIGHWAY 25,BERNIE,MO,63822,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,25401.0,POULTRY FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [145]:
igroup_sic_254_dict['AL']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,HIDDEN ACRES,13892 COUNTY ROAD 87,WOODLAND,AL,36280,25401,POULTRY FARMS,11234001.0,POULTRY HATCHERIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,JEROME KING FARMS,3875 COUNTY ROAD 6,PIEDMONT,AL,36272,25401,POULTRY FARMS,11234001.0,POULTRY HATCHERIES,19101.0,FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ROCK CREEK FARMS,400 COUNTY ROAD 134,TOWN CREEK,AL,35672,201501,POULTRY PROCESSING PLANTS (MFRS),31161501.0,POULTRY PROCESSING,19101.0,FARMS,25401.0,POULTRY FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ROCK CREEK FARMS,400 COUNTY ROAD 134,TOWN CREEK,AL,35672,201501,POULTRY PROCESSING PLANTS (MFRS),31161501.0,POULTRY PROCESSING,19101.0,FARMS,25401.0,POULTRY FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,FOUR H FARMS,2440 COUNTY ROAD 113,CENTRE,AL,35960,25401,POULTRY FARMS,11234001.0,POULTRY HATCHERIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,BREEZY BOTTOM FARM,158 NICHOLS RD,ALTOONA,AL,35952,421304,TRUCKING,48423013.0,SPECIALIZED FREIGHT (EXC USED GDS) TRCKNG LNG-...,19101.0,FARMS,25401.0,POULTRY FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,TYSON FOODS INC,200 CARNES CHAPEL RD,ATTALLA,AL,35954,204803,FEED-MANUFACTURERS,31111903.0,OTHER ANIMAL FOOD MANUFACTURING,514404.0,POULTRY-WHOLESALE,25401.0,POULTRY FARMS,201104.0,MEAT PRODUCTS (MFRS),201501.0,POULTRY PROCESSING PLANTS (MFRS),207702.0,RENDERING COMPANIES (MFRS),7537913.0
7,DALE MURPHY POULTRY,1355 WARRIOR VALLEY RD,ALTOONA,AL,35952,25401,POULTRY FARMS,11234001.0,POULTRY HATCHERIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,CONAGRA POULTRY CO,INDUSTRIAL PARK RD,FORT PAYNE,AL,35967,25401,POULTRY FARMS,11234001.0,POULTRY HATCHERIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7509045.0


In [146]:
igroup_sic_254_dict['LA']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,MAI ANH POULTRY FARM LLC,259 STOREY RD,SIMSBORO,LA,71275,25401,POULTRY FARMS,11234001.0,POULTRY HATCHERIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [147]:
igroup_sic_254_dict['MS']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,SUMMERS POULTRY FARM,945 KOKOMO RD,KOKOMO,MS,39643,201501,POULTRY PROCESSING PLANTS (MFRS),31161501.0,POULTRY PROCESSING,25401.0,POULTRY FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TYSON FOODS INC,855 INDUSTRIAL PARK DR SE,MAGEE,MS,39111,204803,FEED-MANUFACTURERS,31111903.0,OTHER ANIMAL FOOD MANUFACTURING,201501.0,POULTRY PROCESSING PLANTS (MFRS),514404.0,POULTRY-WHOLESALE,25198.0,BROILER FRYER & ROASTER CHICKENS,25401.0,POULTRY FARMS,201104.0,MEAT PRODUCTS (MFRS),7537913.0
2,EVANS POULTRY FARMS,380 SCR 79,MIZE,MS,39116,201501,POULTRY PROCESSING PLANTS (MFRS),31161501.0,POULTRY PROCESSING,25401.0,POULTRY FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ROBERTSON FARMS,12421 ROAD 759,PHILADELPHIA,MS,39350,75102,LIVESTOCK BREEDERS,11521010.0,SUPPORT ACTIVITIES FOR ANIMAL PRODUCTION,25401.0,POULTRY FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PECO FOODS INC,999 HERMAN ALFORD MEMORIAL HWY,PHILADELPHIA,MS,39350,25401,POULTRY FARMS,11234001.0,POULTRY HATCHERIES,514404.0,POULTRY-WHOLESALE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,531052413.0
5,PEDEN POULTRY FARM,10250 ROAD 565,PHILADELPHIA,MS,39350,201501,POULTRY PROCESSING PLANTS (MFRS),31161501.0,POULTRY PROCESSING,25401.0,POULTRY FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [148]:
igroup_sic_254_dict['IN']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,EGG INNOVATIONS,4811 W 100 N,WARSAW,IN,46580,25401,POULTRY FARMS,11234001.0,POULTRY HATCHERIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EGG INNOVATIONS,4811 W 100 N,WARSAW,IN,46580,25401,POULTRY FARMS,11234001.0,POULTRY HATCHERIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,RIVER VIEW FARMS INC,2262 E COUNTY ROAD 500 N,ORLEANS,IN,47452,25401,POULTRY FARMS,11234001.0,POULTRY HATCHERIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,S & G POULTRY FARM,2496 S 400 W,MONROE,IN,46772,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,25401.0,POULTRY FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PERDUE FARMS INC,500 PERDUE RD,VINCENNES,IN,47591,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,201501.0,POULTRY PROCESSING PLANTS (MFRS),25401.0,POULTRY FARMS,NaN,NaN,NaN,NaN,NaN,NaN,7529217.0
5,MINNICH POULTRY,8563 E 300 N,PORTLAND,IN,47371,25401,POULTRY FARMS,11234001.0,POULTRY HATCHERIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [149]:
igroup_sic_254_dict['OH']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,HERTZFELD POULTRY FARMS,15799 MILTON RD,GRAND RAPIDS,OH,43522,19101,FARMS,11199803.0,ALL OTHER MISCELLANEOUS CROP FARMING,25401.0,POULTRY FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MEYER HATCHERY,626 STATE ROUTE 89,POLK,OH,44866,92101,FISHERY CONSULTANTS,11251102.0,FINFISH FARMING & FISH HATCHERIES,25401.0,POULTRY FARMS,549907.0,POULTRY-RETAIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [150]:
igroup_sic_254_dict['AR']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,TYSON FOODS INC,405 S PLEASANT ST,SPRINGDALE,AR,72764,25401,POULTRY FARMS,11234001.0,POULTRY HATCHERIES,201501.0,POULTRY PROCESSING PLANTS (MFRS),201104.0,MEAT PRODUCTS (MFRS),204803.0,FEED-MANUFACTURERS,207702.0,RENDERING COMPANIES (MFRS),209903.0,FOOD PRODUCTS & MANUFACTURERS,7537913.0
1,GEORGE'S INC,402 W ROBINSON AVE,SPRINGDALE,AR,72764,201501,POULTRY PROCESSING PLANTS (MFRS),31161501.0,POULTRY PROCESSING,999966.0,FEDERAL GOVERNMENT CONTRACTORS,25401.0,POULTRY FARMS,NaN,NaN,NaN,NaN,NaN,NaN,512392.0


In [151]:
igroup_sic_254_dict['NC']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,NEWTON'S CHICKEN RANCH,920 GADDY RD S,NEBO,NC,28761,25401,POULTRY FARMS,11234001.0,POULTRY HATCHERIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MOUNTAIRE FARMS-NORTH CAROLINA,4555 US HIGHWAY 421 S,SILER CITY,NC,27344,201501,POULTRY PROCESSING PLANTS (MFRS),31161501.0,POULTRY PROCESSING,25401.0,POULTRY FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,835874538.0
2,CASE FARMS MORGANTON HATCHERY,2102 CONLEY RD,MORGANTON,NC,28655,25401,POULTRY FARMS,11234001.0,POULTRY HATCHERIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4034419.0


In [152]:
igroup_sic_254_dict['GA']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,TYSON FOODS INC,1151 E PINE ST,VIENNA,GA,31092,201501,POULTRY PROCESSING PLANTS (MFRS),31161501.0,POULTRY PROCESSING,514101.0,FOOD PRODUCTS (WHLS),209903.0,FOOD PRODUCTS & MANUFACTURERS,549907.0,POULTRY-RETAIL,549911.0,GAME BIRDS VENISON ETC,25401.0,POULTRY FARMS,7537913.0
1,CLAXTON POULTRY FARMS,8816 US HIGHWAY 301,CLAXTON,GA,30417,201501,POULTRY PROCESSING PLANTS (MFRS),31161501.0,POULTRY PROCESSING,25401.0,POULTRY FARMS,509901.0,EXPORTERS (WHLS),514404.0,POULTRY-WHOLESALE,514403.0,EGGS (WHLS),NaN,NaN,NaN
2,PILGRIM'S PRIDE CORP,136 SWAN BRIDGE RD,TALKING ROCK,GA,30175,25401,POULTRY FARMS,11234001.0,POULTRY HATCHERIES,201501.0,POULTRY PROCESSING PLANTS (MFRS),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,987289857.0


## SIC Code 5144 Exploration: Poultry & Poultry Products

In [153]:
# To see how many matches exist within the dataframe consisting of SIC Codes containing 0762
df_sic_5144 = pd.read_csv("../data/code_5144.csv")
  
igroup_sic_5144_dict, ctrglow_5144_dict = infogroup_counterglow_dict(df_sic_5144, df_counterglow_poultry, states, RADIUS_KM)


IA: Total number of matches: 12
MO: Total number of matches: 10
AL: Total number of matches: 6
MS: Total number of matches: 3
IN: Total number of matches: 4
KY: Total number of matches: 1
AR: Total number of matches: 1
NC: Total number of matches: 1
GA: Total number of matches: 1


In [154]:
igroup_sic_5144_dict = keep_certain_columns(igroup_sic_5144_dict)

#### Explore

In [155]:
igroup_sic_5144_dict['IA']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,VALO BIOMEDIA NORTH AMERICA,21459 OLD HIGHWAY 6,ADEL,IA,50003,283402,POULTRY MEDICINES & VACCINES (MFRS),32541208,PHARMACEUTICAL PREPARATION MANUFACTURING,25401.0,POULTRY FARMS,999966.0,FEDERAL GOVERNMENT CONTRACTORS,545101.0,DAIRY PRODUCTS-RETAIL,514404.0,POULTRY-WHOLESALE,NaN,NaN,843668294.0
1,FREMONT FARMS OF IOWA,3868 110TH ST,MALCOM,IA,50157,514403,EGGS (WHLS),42444001,POULTRY & POULTRY PRODUCT MERCHANT WHOLESALERS,999966.0,FEDERAL GOVERNMENT CONTRACTORS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CENTRUM VALLEY FARMS,2674 HIGHWAY 69,GALT,IA,50101,19101,FARMS,11199803,ALL OTHER MISCELLANEOUS CROP FARMING,514403.0,EGGS (WHLS),541105.0,GROCERS-RETAIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CENTRUM VALLEY FARMS,2674 HIGHWAY 69,GALT,IA,50101,19101,FARMS,11199803,ALL OTHER MISCELLANEOUS CROP FARMING,514403.0,EGGS (WHLS),541105.0,GROCERS-RETAIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,IOWA CAGE FREE,1641 YELLOWSTONE AVE,CLEARFIELD,IA,50840,514403,EGGS (WHLS),42444001,POULTRY & POULTRY PRODUCT MERCHANT WHOLESALERS,541105.0,GROCERS-RETAIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,OSAGE EGG CO,3405 KIRKWOOD AVE,OSAGE,IA,50461,514403,EGGS (WHLS),42444001,POULTRY & POULTRY PRODUCT MERCHANT WHOLESALERS,549914.0,EGGS-RETAIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,SIOUX COUNTY EGG FARM LLP,2716 380TH ST,SIOUX CENTER,IA,51250,19101,FARMS,11199803,ALL OTHER MISCELLANEOUS CROP FARMING,514404.0,POULTRY-WHOLESALE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,E & W POULTRY FARM,2618 340TH ST,ROCK VALLEY,IA,51247,514403,EGGS (WHLS),42444001,POULTRY & POULTRY PRODUCT MERCHANT WHOLESALERS,549914.0,EGGS-RETAIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,SUNRISE FARMS INC,2060 WHITE AVE,HARRIS,IA,51345,514403,EGGS (WHLS),42444001,POULTRY & POULTRY PRODUCT MERCHANT WHOLESALERS,19101.0,FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,FREMONT FARMS,3205 295TH ST,FREMONT,IA,52561,514101,FOOD PRODUCTS (WHLS),42442005,PACKAGED FROZEN FOOD MERCHANT WHOLESALERS,514403.0,EGGS (WHLS),999966.0,FEDERAL GOVERNMENT CONTRACTORS,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [156]:
igroup_sic_5144_dict['MO']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,JOHNSON COUNTY EGG FARM,1275 SE Y HWY,KNOB NOSTER,MO,65336,19101,FARMS,11199803,ALL OTHER MISCELLANEOUS CROP FARMING,514403.0,EGGS (WHLS),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MID STATES SPECIALTY EGGS,30911 HIGHWAY HH,SMITHTON,MO,65350,514403,EGGS (WHLS),42444001,POULTRY & POULTRY PRODUCT MERCHANT WHOLESALERS,545101.0,DAIRY PRODUCTS-RETAIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KNOB NOSTER PULLETS,1251 SE Y HWY,KNOB NOSTER,MO,65336,514403,EGGS (WHLS),42444001,POULTRY & POULTRY PRODUCT MERCHANT WHOLESALERS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,JOHNSON COUNTY EGG FARM,1285 SE Y HWY,KNOB NOSTER,MO,65336,19101,FARMS,11199803,ALL OTHER MISCELLANEOUS CROP FARMING,514403.0,EGGS (WHLS),549914.0,EGGS-RETAIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,LEUSCHEN POULTRY OPERATIONS,1175 DAIRY CIR,CRANE,MO,65633,514402,POULTRY SERVICES (WHLS),42444002,POULTRY & POULTRY PRODUCT MERCHANT WHOLESALERS,25401.0,POULTRY FARMS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,MOARK PRODUCTION,17331 HIGHWAY D,NEOSHO,MO,64850,514403,EGGS (WHLS),42444001,POULTRY & POULTRY PRODUCT MERCHANT WHOLESALERS,549914.0,EGGS-RETAIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,SIMMONS FOODS INC,10700 S STATE HWY,SOUTH WEST CITY,MO,64863,201501,POULTRY PROCESSING PLANTS (MFRS),31161501,POULTRY PROCESSING,514402.0,POULTRY SERVICES (WHLS),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7534076.0
7,SIMMONS FOODS INC,10700 S STATE HWY,SOUTH WEST CITY,MO,64863,201501,POULTRY PROCESSING PLANTS (MFRS),31161501,POULTRY PROCESSING,514402.0,POULTRY SERVICES (WHLS),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7534076.0
8,MOARK PRODUCTION INC,19242 MALLARD DR,NEOSHO,MO,64850,19101,FARMS,11199803,ALL OTHER MISCELLANEOUS CROP FARMING,514403.0,EGGS (WHLS),549914.0,EGGS-RETAIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,TYSON FOODS INC,19597 STATE HWY E,BLOOMFIELD,MO,63825,25401,POULTRY FARMS,11234001,POULTRY HATCHERIES,514404.0,POULTRY-WHOLESALE,201501.0,POULTRY PROCESSING PLANTS (MFRS),201104.0,MEAT PRODUCTS (MFRS),204803.0,FEED-MANUFACTURERS,207702.0,RENDERING COMPANIES (MFRS),7537913.0


In [157]:
igroup_sic_5144_dict['AL']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,NATURES BEST EGG CO,1400 COUNTY ROAD 1568,CULLMAN,AL,35058,514403,EGGS (WHLS),42444001,POULTRY & POULTRY PRODUCT MERCHANT WHOLESALERS,509223.0,WHOLESALERS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NATURES BEST EGG CO,1400 COUNTY ROAD 1568,CULLMAN,AL,35058,514403,EGGS (WHLS),42444001,POULTRY & POULTRY PRODUCT MERCHANT WHOLESALERS,509223.0,WHOLESALERS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,KOCH FOODS INC,764 GEORGE CAGLE DR,COLLINSVILLE,AL,35961,201501,POULTRY PROCESSING PLANTS (MFRS),31161501,POULTRY PROCESSING,514402.0,POULTRY SERVICES (WHLS),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,517549762.0
3,KOCH FOODS INC,764 GEORGE CAGLE DR,COLLINSVILLE,AL,35961,201501,POULTRY PROCESSING PLANTS (MFRS),31161501,POULTRY PROCESSING,514402.0,POULTRY SERVICES (WHLS),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,517549762.0
4,KOCH FOODS INC,764 GEORGE CAGLE DR,COLLINSVILLE,AL,35961,201501,POULTRY PROCESSING PLANTS (MFRS),31161501,POULTRY PROCESSING,514402.0,POULTRY SERVICES (WHLS),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,517549762.0
5,TYSON FOODS INC,200 CARNES CHAPEL RD,ATTALLA,AL,35954,204803,FEED-MANUFACTURERS,31111903,OTHER ANIMAL FOOD MANUFACTURING,514404.0,POULTRY-WHOLESALE,25401.0,POULTRY FARMS,201104.0,MEAT PRODUCTS (MFRS),201501.0,POULTRY PROCESSING PLANTS (MFRS),207702.0,RENDERING COMPANIES (MFRS),7537913.0


In [158]:
igroup_sic_5144_dict['MS']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,TOTAL POULTRY SVC LLC,130 STOVALL RD # 2,KOKOMO,MS,39643,514402,POULTRY SERVICES (WHLS),42444002,POULTRY & POULTRY PRODUCT MERCHANT WHOLESALERS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TYSON FOODS INC,855 INDUSTRIAL PARK DR SE,MAGEE,MS,39111,204803,FEED-MANUFACTURERS,31111903,OTHER ANIMAL FOOD MANUFACTURING,201501.0,POULTRY PROCESSING PLANTS (MFRS),514404.0,POULTRY-WHOLESALE,25198.0,BROILER FRYER & ROASTER CHICKENS,25401.0,POULTRY FARMS,201104.0,MEAT PRODUCTS (MFRS),7537913.0
2,PECO FOODS INC,999 HERMAN ALFORD MEMORIAL HWY,PHILADELPHIA,MS,39350,25401,POULTRY FARMS,11234001,POULTRY HATCHERIES,514404.0,POULTRY-WHOLESALE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,531052413.0


In [159]:
igroup_sic_5144_dict['IN']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,EGGPRESS LLC,4217 W OLD ROAD 30,WARSAW,IN,46580,514403,EGGS (WHLS),42444001,POULTRY & POULTRY PRODUCT MERCHANT WHOLESALERS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CREIGHTON BROTHERS LLC,4217 W OLD ROAD 30,WARSAW,IN,46580,514404,POULTRY-WHOLESALE,42444003,POULTRY & POULTRY PRODUCT MERCHANT WHOLESALERS,999966.0,FEDERAL GOVERNMENT CONTRACTORS,25198.0,BROILER FRYER & ROASTER CHICKENS,201501.0,POULTRY PROCESSING PLANTS (MFRS),209998.0,FOOD PREPARATIONS NEC (MFRS),519102.0,FARM SUPPLIES (WHLS),NaN
2,WABASH VALLEY PRODUCE INC,4886 E 450N,DUBOIS,IN,47527,204803,FEED-MANUFACTURERS,31111903,OTHER ANIMAL FOOD MANUFACTURING,519112.0,FEED-DEALERS (WHLS),514403.0,EGGS (WHLS),549914.0,EGGS-RETAIL,999966.0,FEDERAL GOVERNMENT CONTRACTORS,NaN,NaN,NaN
3,WEAVER AG & LAWN EQUIPMENT LLC,905 NELSONS PKWY,WAKARUSA,IN,46573,78206,LAWN & GROUNDS MAINTENANCE,56173011,LANDSCAPING SERVICES,526109.0,LAWN MOWERS,508310.0,FARM EQUIPMENT (WHLS),526101.0,LAWN & GARDEN EQUIP & SUPPLIES-RETAIL,508331.0,AGRICULTURAL IMPLEMENTS & SUPPLIES-WHLS,505144.0,STEEL PRODUCTS (WHLS),NaN


In [160]:
igroup_sic_5144_dict['KY']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,CAL-MAINE FOODS INC,9729 GUTHRIE RD,GUTHRIE,KY,42234,514403,EGGS (WHLS),42444001,POULTRY & POULTRY PRODUCT MERCHANT WHOLESALERS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,431481290.0


In [161]:
igroup_sic_5144_dict['AR']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,PREFERRED POULTRY SUPPLY LLC,2285 E PRIDEMORE DR,LINCOLN,AR,72744,514402,POULTRY SERVICES (WHLS),42444002,POULTRY & POULTRY PRODUCT MERCHANT WHOLESALERS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [162]:
igroup_sic_5144_dict['NC']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,SIMPSON'S EGGS INC,5015 E HIGHWAY 218,MONROE,NC,28110,514403,EGGS (WHLS),42444001,POULTRY & POULTRY PRODUCT MERCHANT WHOLESALERS,549914.0,EGGS-RETAIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [163]:
igroup_sic_5144_dict['GA']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,CLAXTON POULTRY FARMS,8816 US HIGHWAY 301,CLAXTON,GA,30417,201501,POULTRY PROCESSING PLANTS (MFRS),31161501,POULTRY PROCESSING,25401.0,POULTRY FARMS,509901.0,EXPORTERS (WHLS),514404.0,POULTRY-WHOLESALE,514403.0,EGGS (WHLS),NaN,NaN,NaN


## SIC Code 5154 Exploration: Livestock

In [164]:
# To see how many matches exist within the dataframe consisting of SIC Codes containing 0762
df_sic_5154 = pd.read_csv("../data/code_5154.csv")
  
igroup_sic_5154_dict, ctrglow_5154_dict = infogroup_counterglow_dict(df_sic_5154, df_counterglow_poultry, states, RADIUS_KM)


IA: Total number of matches: 3
AL: Total number of matches: 3
IN: Total number of matches: 2
NC: Total number of matches: 2


In [166]:
igroup_sic_5154_dict = keep_certain_columns(igroup_sic_5154_dict)

#### Explore

In [167]:
igroup_sic_5154_dict['IA']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,TRI-STATE LIVESTOCK,1375 N MAIN AVE,SIOUX CENTER,IA,51250,515402,LIVESTOCK AUCTION MARKETS (WHLS),42452004,LIVESTOCK MERCHANT WHOLESALERS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GRAHAM POLLED HEREFORDS,2509 155TH AVE,BENTON,IA,50835,515406,CATTLE COMPANIES (WHLS),42452001,LIVESTOCK MERCHANT WHOLESALERS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TLC LIVESTOCK INC,3804 40TH AVE,BANCROFT,IA,50517,515403,LIVESTOCK-DEALERS (WHLS),42452008,LIVESTOCK MERCHANT WHOLESALERS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [168]:
igroup_sic_5154_dict['AL']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,ARAB LIVESTOCK MARKET INC,2345 US HIGHWAY 231,ARAB,AL,35016,515402,LIVESTOCK AUCTION MARKETS (WHLS),42452004,LIVESTOCK MERCHANT WHOLESALERS,478904.0,STOCK YARDS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ROCKING W LIVESTOCK EQPT SALES,600 N JOHNSON CHAPEL RD,DANVILLE,AL,35619,515403,LIVESTOCK-DEALERS (WHLS),42452008,LIVESTOCK MERCHANT WHOLESALERS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MID STATE STOCKYARDS LLP,8415 STATE HIGHWAY 97 S,LETOHATCHEE,AL,36047,478904,STOCK YARDS,48821007,SUPPORT ACTIVITIES FOR RAIL TRANSPORTATION,515402.0,LIVESTOCK AUCTION MARKETS (WHLS),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [169]:
igroup_sic_5154_dict['IN']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,BROWN BARN LIVESTOCK,5502 S 1000 W,MENTONE,IN,46539,515403,LIVESTOCK-DEALERS (WHLS),42452008,LIVESTOCK MERCHANT WHOLESALERS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,WOLF CO-OP,109 UNION ST,WOLCOTTVILLE,IN,46795,653107,COOPERATIVES,81399002,OTHER SIMILAR ORGANIZATIONS,519112.0,FEED-DEALERS (WHLS),515403.0,LIVESTOCK-DEALERS (WHLS),NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [170]:
igroup_sic_5154_dict['NC']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,J-D FARMS LLC,170 COGNAC RD,MARSTON,NC,28363,75102,LIVESTOCK BREEDERS,11521010,SUPPORT ACTIVITIES FOR ANIMAL PRODUCTION,519119.0,LIVESTOCK EQUIPMENT & SUPPLIES (WHLS),515402.0,LIVESTOCK AUCTION MARKETS (WHLS),NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,FIGUEROA LIVESTOCK INC,5020 SOUTHBROOK DR,KINGS MOUNTAIN,NC,28086,515403,LIVESTOCK-DEALERS (WHLS),42452008,LIVESTOCK MERCHANT WHOLESALERS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## SIC Code 0251: Broiler, Fryer, and Roaster Chickens

In [11]:
# To see how many matches exist within the dataframe consisting of SIC Codes containing 0762
df_sic_0251 = pd.read_csv("../data/code_0251.csv")
  
igroup_sic_0251_dict, ctrglow_0251_dict = infogroup_counterglow_dict(df_sic_0251, df_counterglow_poultry, states, RADIUS_KM)


MO: Total number of matches: 4
AL: Total number of matches: 1
MS: Total number of matches: 1
IN: Total number of matches: 1
NC: Total number of matches: 1
GA: Total number of matches: 1


In [12]:
igroup_sic_0251_dict = keep_certain_columns(igroup_sic_0251_dict)

#### Explore

In [13]:
igroup_sic_0251_dict['MO']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,TRIPLE P RANCH & EQUIPMENT LLC,22211 FIVE BRIDGES AVE,COLE CAMP,MO,65325,25198,BROILER FRYER & ROASTER CHICKENS,11232001,BROILERS & OTHER MEAT TYPE CHICKEN PRODUCTION,508303.0,POULTRY EQUIPMENT & SUPPLIES (WHLS),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TRIPLE P RANCH & EQUIPMENT LLC,22211 FIVE BRIDGES AVE,COLE CAMP,MO,65325,25198,BROILER FRYER & ROASTER CHICKENS,11232001,BROILERS & OTHER MEAT TYPE CHICKEN PRODUCTION,508303.0,POULTRY EQUIPMENT & SUPPLIES (WHLS),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TRIPLE P RANCH & EQUIPMENT LLC,22211 FIVE BRIDGES AVE,COLE CAMP,MO,65325,25198,BROILER FRYER & ROASTER CHICKENS,11232001,BROILERS & OTHER MEAT TYPE CHICKEN PRODUCTION,508303.0,POULTRY EQUIPMENT & SUPPLIES (WHLS),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TRIPLE P RANCH & EQUIPMENT LLC,22211 FIVE BRIDGES AVE,COLE CAMP,MO,65325,25198,BROILER FRYER & ROASTER CHICKENS,11232001,BROILERS & OTHER MEAT TYPE CHICKEN PRODUCTION,508303.0,POULTRY EQUIPMENT & SUPPLIES (WHLS),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
igroup_sic_0251_dict['AL']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,WOODHAM FARM,2501 COUNTY ROAD 53,CLAYTON,AL,36016,25298,CHICKEN EGGS,11231001,CHICKEN EGG PRODUCTION,25198.0,BROILER FRYER & ROASTER CHICKENS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
igroup_sic_0251_dict['MS']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,TYSON FOODS INC,855 INDUSTRIAL PARK DR SE,MAGEE,MS,39111,204803,FEED-MANUFACTURERS,31111903,OTHER ANIMAL FOOD MANUFACTURING,201501.0,POULTRY PROCESSING PLANTS (MFRS),514404.0,POULTRY-WHOLESALE,25198.0,BROILER FRYER & ROASTER CHICKENS,25401.0,POULTRY FARMS,201104.0,MEAT PRODUCTS (MFRS),7537913.0


In [16]:
igroup_sic_0251_dict['IN']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,CREIGHTON BROTHERS LLC,4217 W OLD ROAD 30,WARSAW,IN,46580,514404,POULTRY-WHOLESALE,42444003,POULTRY & POULTRY PRODUCT MERCHANT WHOLESALERS,999966.0,FEDERAL GOVERNMENT CONTRACTORS,25198.0,BROILER FRYER & ROASTER CHICKENS,201501.0,POULTRY PROCESSING PLANTS (MFRS),209998.0,FOOD PREPARATIONS NEC (MFRS),519102.0,FARM SUPPLIES (WHLS),NaN


In [17]:
igroup_sic_0251_dict['NC']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,REYNOLDS FARMS,7518 CARL POLK RD,MONROE,NC,28110,25198,BROILER FRYER & ROASTER CHICKENS,11232001,BROILERS & OTHER MEAT TYPE CHICKEN PRODUCTION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
igroup_sic_0251_dict['GA']

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),PARENT NUMBER
0,MERCER FARMS,823 GEORGE FLOWERS RD,NORMAN PARK,GA,31771,25198,BROILER FRYER & ROASTER CHICKENS,11232001,BROILERS & OTHER MEAT TYPE CHICKEN PRODUCTION,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
